# Heights problem with restricted prior on mu.

Result is not conform cmdstan result

In [1]:
using StatisticalRethinking
using DynamicHMC, TransformVariables, LogDensityProblems, MCMCDiagnostics
using Parameters, ForwardDiff

ProjDir = rel_path("..", "scripts", "04")
cd(ProjDir)

Import the dataset.

In [2]:
howell1 = CSV.read(rel_path("..", "data", "Howell1.csv"), delim=';')
df = convert(DataFrame, howell1);

Use only adults and standardize

In [3]:
df2 = filter(row -> row[:age] >= 18, df);

Show the first six rows of the dataset.

In [4]:
first(df2, 6)

,height,weight,age,male
,Float64⍰,Float64⍰,Float64⍰,Int64⍰
1,151.765,47.8256,63.0,1
2,139.7,36.4858,63.0,0
3,136.525,31.8648,65.0,0
4,156.845,53.0419,41.0,1
5,145.415,41.2769,51.0,0
6,163.83,62.9926,35.0,1


No covariates, just height observations.

In [5]:
struct ConstraintHeightsProblem{TY <: AbstractVector}
    "Observations."
    y::TY
end;

Very constraint prior on μ. Flat σ.

In [6]:
function (problem::ConstraintHeightsProblem)(θ)
    @unpack y = problem   # extract the data
    @unpack μ, σ = θ
    loglikelihood(Normal(μ, σ), y) + logpdf(Normal(178, 0.1), μ) +
    logpdf(Uniform(0, 50), σ)
end;

Define problem with data and inits.

In [7]:
obs = convert(Vector{Float64}, df2[:height])
p = ConstraintHeightsProblem(obs);
p((μ = 178, σ = 5.0))

-5169.102069832888

Wrap the problem with a transformation, then use Flux for the gradient.

In [8]:
P = TransformedLogDensity(as((μ  = as(Real, 100, 250), σ = asℝ₊)), p)
∇P = ADgradient(:ForwardDiff, P);

FSample from the posterior.

In [9]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000);

MCMC, adapting ϵ (75 steps)
0.0015 s/step ...done
MCMC, adapting ϵ (25 steps)
0.00022 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0028 s/step ...done
MCMC, adapting ϵ (100 steps)
0.00013 s/step ...done
MCMC, adapting ϵ (200 steps)
9.3e-5 s/step ...done
MCMC, adapting ϵ (400 steps)
7.8e-5 s/step ...done
MCMC, adapting ϵ (50 steps)
5.4e-5 s/step ...done
MCMC (1000 steps)
8.3e-5 s/step ...done


Undo the transformation to obtain the posterior from the chain.

In [10]:
posterior = TransformVariables.transform.(Ref(∇P.transformation), get_position.(chain));

Extract the parameter posterior means: `μ`,

In [11]:
posterior_μ = mean(first, posterior)

177.8630467747113

Extract the parameter posterior means: `μ`,

In [12]:
posterior_σ = mean(last, posterior)

24.544670647112103

Effective sample sizes (of untransformed draws)

In [13]:
ess = mapslices(effective_sample_size,
                get_position_matrix(chain); dims = 1)
# NUTS-specific statistics
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.92, min/25%/median/75%/max: 0.34 0.89 0.96 0.99 1.0
  termination: AdjacentTurn => 35% DoubledTurn => 65%
  depth: 1 => 16% 2 => 66% 3 => 17% 4 => 0%


cmdstan result

In [14]:
cmdstan_result = "
Iterations = 1:1000
Thinning interval = 1
Chains = 1,2,3,4
Samples per chain = 1000

Empirical Posterior Estimates:
         Mean         SD       Naive SE       MCSE      ESS
sigma  24.604616 0.946911707 0.0149719887 0.0162406632 1000
   mu 177.864069 0.102284043 0.0016172527 0.0013514459 1000

Quantiles:
         2.5%       25.0%     50.0%     75.0%     97.5%
sigma  22.826377  23.942275  24.56935  25.2294  26.528368
   mu 177.665000 177.797000 177.86400 177.9310 178.066000
";

Extract the parameter posterior means: `β`,

In [15]:
[posterior_μ, posterior_σ]

2-element Array{Float64,1}:
 177.8630467747113  
  24.544670647112103

end of m4.5d.jl#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*